In [5]:
import datetime
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import re



In [15]:
url = 'https://docs.google.com/spreadsheets/d/1aCENVOk-wroyW4-YS4OgtTTqr3rA-T9CZOjCQgALgSE/export?format=xlsx'
xls = pd.ExcelFile(url)

exclude = {"_Competition Factors", "_JPAR Ratings", "_Latest JPAR Ratings", "_Histograms", "_UTILITY FUNCTIONS"}
sheets_to_read = [s for s in xls.sheet_names if s not in exclude]

dfs = {
    name: xls.parse(name, dtype={'Time': str})
    for name in sheets_to_read
}


In [16]:
for name, df in dfs.items():
    fullname = df['Event'].iloc[0]
    df['Full_Event'] = fullname
    print(name)
    print(df)

W19Ind
     Rank                              Name      Time  Remaining        PPM  \
0       1                  Jana Ondroušková  00:46:35        NaN  10.733453   
1       2                 Mariusz Ślizewski  00:52:25        NaN   9.538951   
2       3                 Kateřina Klinková  00:52:50        NaN   9.463722   
3       4                       Sarah Mills  00:55:24        NaN   9.025271   
4       5                   Ana Gil Luciano  00:55:38        NaN   8.987418   
..    ...                               ...       ...        ...        ...   
194   195                     Joseph Kasozi  02:00:00      313.0   1.558333   
195   196      Agustín De la Rosa Hernández  02:00:00      314.0   1.550000   
196   197  Maria Alexandra Duarte Rodrigues  02:00:00      324.0   1.466667   
197   198                     Arnold Jacobs  02:00:00      327.0   1.441667   
198   199                  Dora Maria Polle  02:00:00      364.0   1.133333   

     Relative Rank       Date  Pieces       

In [18]:
for name, df in dfs.items():
    df['Event'] = name

combined_df = pd.concat(dfs.values(), ignore_index=True)
combined_df = combined_df.drop(columns=['Unnamed: 11', 'Unnamed: 12'], errors='ignore')

def time_to_seconds(time_str):
    try:
        h, m, s = map(int, time_str.split(':'))
        return h * 3600 + m * 60 + s
    except Exception:
        return np.nan

combined_df['time_in_seconds'] = combined_df['Time'].apply(time_to_seconds)

def clean_remaining(val):
    if pd.isna(val):
        return 0
    if isinstance(val, str) and re.fullmatch(r"\d{1,2}:\d{2}:\d{2}", val):
        return 0
    return val

combined_df["Remaining"] = combined_df["Remaining"].apply(clean_remaining)

combined_df['time_penalty'] = (((500 - combined_df['Remaining']) / combined_df['time_in_seconds'])**(-1)) * combined_df['Remaining']
combined_df['corrected_time'] = combined_df['time_penalty'] + combined_df['time_in_seconds']

## DATA CLEANING
combined_df['Name'] = combined_df['Name'].str.strip()
combined_df['Pieces'] = pd.to_numeric(combined_df['Pieces'],errors='coerce')

In [19]:
combined_df

,Rank,Name,Time,Remaining,PPM,Relative Rank,Date,Pieces,Event,Players,Spread Diff,Full_Event,time_in_seconds,time_penalty,corrected_time
0,1.0,Jana Ondroušková,00:46:35,0.0,10.733453,2.528086,2019-09-24,500.0,W19Ind,1,3.102035,Worlds 2019 - Individual,2795.0,0.000000,2795.000000
1,2.0,Mariusz Ślizewski,00:52:25,0.0,9.538951,2.246741,2019-09-24,500.0,W19Ind,1,2.492723,Worlds 2019 - Individual,3145.0,0.000000,3145.000000
2,3.0,Kateřina Klinková,00:52:50,0.0,9.463722,2.229022,2019-09-24,500.0,W19Ind,1,2.454349,Worlds 2019 - Individual,3170.0,0.000000,3170.000000
3,4.0,Sarah Mills,00:55:24,0.0,9.025271,2.125752,2019-09-24,500.0,W19Ind,1,2.230696,Worlds 2019 - Individual,3324.0,0.000000,3324.000000
4,5.0,Ana Gil Luciano,00:55:38,0.0,8.987418,2.116836,2019-09-24,500.0,W19Ind,1,2.211388,Worlds 2019 - Individual,3338.0,0.000000,3338.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23612,27.0,Ana Guillén García,02:00:00,195.0,2.541667,0.592971,2025-02-23,500.0,ae326,1,-1.181252,III Concurso de puzzles de Viveiro,7200.0,4603.278689,11803.278689
23613,28.0,Lorena Barreno De Pablos,02:00:00,202.0,2.483333,0.579362,2025-02-23,500.0,ae326,1,-1.220247,III Concurso de puzzles de Viveiro,7200.0,4880.536913,12080.536913
23614,29.0,Mª Carmen Martínez Rodríguez,02:00:00,231.0,2.241667,0.522981,2025-02-23,500.0,ae326,1,-1.381802,III Concurso de puzzles de Viveiro,7200.0,6182.899628,13382.899628
23615,30.0,Rodrigo Jurado Villalobos,02:00:00,233.0,2.225,0.519092,2025-02-23,500.0,ae326,1,-1.392943,III Concurso de puzzles de Viveiro,7200.0,6283.146067,13483.146067


In [21]:
combined_df.to_pickle('./data/250519_scape_with_event_names.pkl')

In [2]:
df

,Rank,Name,Time,Remaining,PPM,Relative Rank,Date,Pieces,Event,Players,Spread Diff,time_in_seconds,time_penalty,corrected_time
0,1.0,Jana Ondroušková,00:46:35,0.0,10.733453,2.528086,2019-09-24,500,W19Ind,1,3.102035,2795.0,0.000000,2795.000000
1,2.0,Mariusz Ślizewski,00:52:25,0.0,9.538951,2.246741,2019-09-24,500,W19Ind,1,2.492723,3145.0,0.000000,3145.000000
2,3.0,Kateřina Klinková,00:52:50,0.0,9.463722,2.229022,2019-09-24,500,W19Ind,1,2.454349,3170.0,0.000000,3170.000000
3,4.0,Sarah Mills,00:55:24,0.0,9.025271,2.125752,2019-09-24,500,W19Ind,1,2.230696,3324.0,0.000000,3324.000000
4,5.0,Ana Gil Luciano,00:55:38,0.0,8.987418,2.116836,2019-09-24,500,W19Ind,1,2.211388,3338.0,0.000000,3338.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23612,27.0,Ana Guillén García,02:00:00,195.0,2.541667,0.592971,2025-02-23,500,ae326,1,-1.181252,7200.0,4603.278689,11803.278689
23613,28.0,Lorena Barreno De Pablos,02:00:00,202.0,2.483333,0.579362,2025-02-23,500,ae326,1,-1.220247,7200.0,4880.536913,12080.536913
23614,29.0,Mª Carmen Martínez Rodríguez,02:00:00,231.0,2.241667,0.522981,2025-02-23,500,ae326,1,-1.381802,7200.0,6182.899628,13382.899628
23615,30.0,Rodrigo Jurado Villalobos,02:00:00,233.0,2.225,0.519092,2025-02-23,500,ae326,1,-1.392943,7200.0,6283.146067,13483.146067


In [6]:
df['time_in_seconds'].argmin()

7173

In [7]:
df.iloc[7173]


Rank                               1.0
Name                       Kate Mixson
Time                          00:18:16
Remaining                          0.0
PPM                          16.423358
Relative Rank                  1.87865
Date               2024-03-09 00:00:00
Pieces                             300
Event                            sp182
Players                              1
Spread Diff                   2.790583
time_in_seconds                 1096.0
time_penalty                       0.0
corrected_time                  1096.0
Name: 7173, dtype: object

In [16]:
df['Pieces'] = pd.to_numeric(df['Pieces'], errors='coerce')

In [1]:
df['Pieces'].sum()

NameError: name 'df' is not defined

In [6]:
df['Pieces'].dropna()

0        500
1        500
2        500
3        500
4        500
        ... 
23612    500
23613    500
23614    500
23615    500
23616    500
Name: Pieces, Length: 23617, dtype: object